In [1]:
import numpy as np
import torch 
from torch.nn.functional import relu

from sklearn.svm import SVC
from sklearn.datasets import load_iris,load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import pennylane as qml
from pennylane.templates import AngleEmbedding, StronglyEntanglingLayers
from pennylane.operation import Tensor

import matplotlib.pyplot as plt

np.random.seed(42)

In [2]:
X, y = load_iris(return_X_y=True)

# pick inputs and labels from the first two classes only,
# corresponding to the first 100 samples
X = X[:100]
y = y[:100]

# scaling the inputs is important since the embedding we use is periodic
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)

# scaling the labels to -1, 1 is important for the SVM and the
# definition of a hinge loss
y_scaled = 2 * (y - 0.5)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled)

In [3]:
def circuit(feature_vector,feature_vector_2,length):
    qml.AngleEmbedding(features=feature_vector, wires=range(length))
    qml.adjoint(qml.AngleEmbedding(features=feature_vector_2, wires=range(length)))
    return qml.probs(wires=range(length))
n_qubits = len(X_train[0])
dev_kernel = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev_kernel, interface="autograd")
def kernel(x1, x2):
    """The quantum kernel."""
    u_1 = qml.matrix(circuit)(x1,x2,len(x1))
    u_2 = u_1.conjugate().transpose()
    projector = u_1+u_2
    return qml.expval(qml.Hermitian(projector,wires=range(n_qubits)))
def kernel_matrix(A, B):
    """Compute the matrix whose entries are the kernel
       evaluated on pairwise data from sets A and B."""
    return np.array([[kernel(a, b) for b in B] for a in A])
def qsvm(Xtrain,ytrain):
    svm = SVC(kernel=kernel_matrix).fit(Xtrain, ytrain)
    return svm

In [4]:
svm = qsvm(X_train,y_train)

In [5]:
predictions = svm.predict(X_test)
accuracy_score(predictions, y_test)

1.0